In [2]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
from cassandra.cluster import Cluster
import pandas as pd
import json

In [81]:
consumer = KafkaConsumer(
    'Amazon_Stock_Daily_Data',
     bootstrap_servers=['localhost:9092'], 
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [29]:
#for con in consumer:
#    print(con.value)

{'Unnamed: 0': 5188, 'Date': '2017-12-26', 'Open': 58.417999267578125, 'High': 58.91600036621094, 'Low': 58.02750015258789, 'Close': 58.8380012512207, 'Volume': 40104000}
{'Unnamed: 0': 5823, 'Date': '2020-07-07', 'Open': 152.92750549316406, 'High': 153.4774932861328, 'Low': 149.5, 'Close': 150.00599670410156, 'Volume': 105150000}
{'Unnamed: 0': 4375, 'Date': '2014-10-03', 'Open': 16.01799964904785, 'High': 16.257999420166016, 'Low': 15.976499557495115, 'Close': 16.136999130249023, 'Volume': 60748000}
{'Unnamed: 0': 5820, 'Date': '2020-07-01', 'Open': 137.89950561523438, 'High': 144.75, 'Low': 137.6999969482422, 'Close': 143.93499755859375, 'Volume': 127268000}
{'Unnamed: 0': 2808, 'Date': '2008-07-15', 'Open': 3.2934999465942383, 'High': 3.430999994277954, 'Low': 3.149499893188477, 'Close': 3.3515000343322754, 'Volume': 226842000}
{'Unnamed: 0': 5419, 'Date': '2018-11-26', 'Open': 76.94999694824219, 'High': 79.2405014038086, 'Low': 76.21099853515625, 'Close': 79.06649780273438, 'Volum

KeyboardInterrupt: 

In [17]:
# Connect to Cassandra
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [101]:
# Create Keyspace
session.execute("""
CREATE KEYSPACE IF NOT EXISTS stellar
  WITH REPLICATION = { 
   'class' : 'SimpleStrategy', 
   'replication_factor' : 3
  };
""")

In [103]:
# Check Keyspace
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")
for row in rows:
    print(row.keyspace_name)

system_auth
system_schema
stellar
system_distributed
system
system_traces


In [104]:
# Use Keyspace
session.set_keyspace('stellar')

In [106]:
# Query to drop table
#query = f"DROP TABLE stellar.Amazon_Stock_Daily"

# Execute query
#session.execute(query)

In [107]:
# create table
session.execute("""
CREATE TABLE IF NOT EXISTS Amazon_Stock_Daily (
                Date date,
                Open decimal,
                High decimal,
                Low decimal,
                Close decimal,
                Volume decimal,
                PRIMARY KEY (Date)
            );
""")

In [108]:
insert_query = session.prepare(
    f"INSERT INTO Amazon_Stock_Daily (date, open, high, low, close, volume) VALUES (?, ?, ?, ?, ?, ?)"
)

In [109]:
# insert data
for message in consumer:
    data = message.value  # Information received from KafkaConsumer
    session.execute(insert_query, (
        data['Date'],
        data['Open'],
        data['High'],
        data['Low'],
        data['Close'],
        data['Volume']
    ))
    print(f"Inserted data for Date: {data['Date']}")

Inserted data for Date: 2021-10-07
Inserted data for Date: 2013-12-16
Inserted data for Date: 2011-12-13
Inserted data for Date: 2018-03-15
Inserted data for Date: 2019-09-16
Inserted data for Date: 2017-02-10
Inserted data for Date: 2009-01-27
Inserted data for Date: 2016-04-18
Inserted data for Date: 2014-08-29
Inserted data for Date: 1998-11-02
Inserted data for Date: 2002-02-08
Inserted data for Date: 2003-10-02
Inserted data for Date: 2006-09-29
Inserted data for Date: 2005-08-01
Inserted data for Date: 2013-12-06
Inserted data for Date: 2010-05-05
Inserted data for Date: 2005-06-06
Inserted data for Date: 2021-04-28
Inserted data for Date: 2017-09-07
Inserted data for Date: 1999-10-26
Inserted data for Date: 2021-08-26
Inserted data for Date: 1997-05-23
Inserted data for Date: 2004-02-04
Inserted data for Date: 2017-10-23
Inserted data for Date: 2001-01-22
Inserted data for Date: 2002-11-27
Inserted data for Date: 2005-02-01
Inserted data for Date: 2023-09-21
Inserted data for Da

KeyboardInterrupt: 

In [111]:
# Check data
#r = session.execute("SELECT * FROM Amazon_Stock_Daily")
#df = pd.DataFrame(r)
#df

,date,close,high,low,open,volume
0,2020-09-18,147.7454986572265625,151.8899993896484375,145.2769927978515625,151.587005615234375,177852000
1,2014-12-01,16.299999237060546875,17.0319995880126953125,16.2964992523193359375,16.9060001373291015625,98898000
2,2002-05-07,0.805499970912933349609375,0.81749999523162841796875,0.801500022411346435546875,0.805499970912933349609375,110874000
3,2000-07-27,1.56875002384185791015625,1.60000002384185791015625,1.4874999523162841796875,1.57187497615814208984375,471586000
4,2010-09-22,7.59149980545043856494658029987476766109466552...,7.6350002288818359375,7.4549999237060546875,7.492000102996826171875,132248000
...,...,...,...,...,...,...
684,2006-11-09,1.94200003147125244140625,1.98849999904632568359375,1.9405000209808349609375,1.97500002384185791015625,116142000
685,2009-12-08,6.705500125885009765625,6.80399990081787109375,6.643499851226806640625,6.715000152587890625,160056000
686,2006-02-06,1.89750003814697287829460492503130808472633361...,1.9170000553131103515625,1.88849997520446799548210492503130808472633361...,1.9170000553131103515625,170064000
687,2021-12-27,169.66949462890625,172.9429931640625,169.2154998779296875,171.0370025634765625,58688000
